# Выполнено Бадиковым Вадимом Радиковичем
vk = vadyushaa,

telegam = sheduele

### Оренбургский Государственный Университет 2022
### 20ИВТ(ПОВТ)

# Начало

In [1]:
import string
import numpy as np
import pandas as pd # для начала используем один импорт 
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
positive = pd.DataFrame
negative = pd.DataFrame
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])

In [3]:
def labeler (comment, label):
    i = 0
    output = []
    while i < len(comment):
        output.append(label)
        i+=1
    return output

In [4]:
def no_RT(test): # Убирает все RT 
    output = []
    i = 0
    u = 0
    test_ln = len(test) - 1
    count_sp = 0
    while True:
        
        jay = test[i]
        if jay[0] == 'R' and jay[1] == 'T':
            
            while u < len(jay):
                if jay[u] == ' ':
                    count_sp += 1
                if count_sp == 2:
                    plus = u+1
                    output.append(jay[plus:])
                    break
                u += 1
            u = 0
            count_sp = 0
        else:
            output.append(jay)
        i += 1
        if i == test_ln:
            break
    return output

In [5]:
def no_dog (test): # убирает все упоминания из текста
    output = []
    u = 0
    gate = False
    was_gate = False
    for i in test:
        while u < len(i):
            if was_gate == False and i[u] == "@":
                gate = True
                was_gate = True
                
            if gate == True and i[u] == " ":
                plus = u+1
                output.append(i[plus:])
                gate = False
                break
            u +=1
        u = 0
        if was_gate == False:
            output.append(i)
        was_gate = False
    return output

In [6]:
def no_links (test):
    output = []
    u = 0
    one_link = 0
    gate = False
    for i in test:
        while u < len(i):
            if one_link == 0 and len(i) - u > 3 and i[u] == 'h' and i[u+1] == 't' and i[u+2] == 't':
                gate = True
                one_link = 1
                output.append(i[:u])
            u +=1
        u = 0
        if gate == False:
            output.append(i)
        gate = False
        one_link = 0
    return output

In [7]:
def sorter(test): # Функция сортировки текста по длине комментария (len(comment) > 10)
    i = 0
    output = []
    while i < len(test):
        splitted = test[i].split(" ")
        if len(splitted) > 9:
            output.append(test[i])
        i+=1
    return output

In [8]:
neg = negative["text"].values
pos = positive["text"].values

In [9]:
neg = no_RT(neg)
neg = no_dog(neg)
neg = no_dog(neg)
neg = no_dog(neg)
neg = no_links(neg)
#neg = sorter(neg)
# использование 3-х функций no_dog объясняется тем, что в некоторых текстах, 
# встречается многократное использование

In [10]:
pos = no_RT(pos)
pos = no_dog(pos)
pos = no_dog(pos)
pos = no_dog(pos)
pos = no_links(pos)
#pos = sorter(pos)
# использование 3-х функций no_dog объясняется тем, что в некоторых текстах, 
# встречается многократное использование знака @

In [11]:
print ( " Positive - " + str(len(pos)) + "\n Negative - " + str(len(neg)))

 Positive - 114707
 Negative - 111852


In [12]:
labels = labeler(pos,0) + labeler(neg,1)

In [13]:
df = pos + neg

In [14]:
df = np.array(df)

# Подготовка текста

In [15]:
import nltk
from nltk import word_tokenize
nltk.download("punkt")
nltk.download('stopwords')
from nltk.corpus import stopwords 
from nltk.stem import SnowballStemmer
from sklearn.pipeline import Pipeline
import string
import numpy as np

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Vadim\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Vadim\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
ru_sw = stopwords.words("russian") # ru stop words
snowball = SnowballStemmer(language = "russian")# stemming

def token_all_proccesing(df: str, remove_stop_words: bool = True): # func
    tokens = word_tokenize(df, language="russian") # tokenize
    tokens = [i for i in tokens if i not in string.punctuation] # tokenize w/o punctuation
    if remove_stop_words:
        tokens = [i for i in tokens if i not in ru_sw]# remove ru stopword
    tokens = [snowball.stem(i) for i in tokens] # snowball stemming
    return tokens

In [17]:
from sklearn.utils import shuffle

In [18]:
texts, labelz = shuffle(df, labels, random_state=0)

# Создание модели (лог. регрессия)

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [20]:
model_pipeline = Pipeline([("vectorizer",TfidfVectorizer(tokenizer= lambda x:token_all_proccesing(x, remove_stop_words= True)))
                  ,("model",LogisticRegression(random_state = 228) )])

In [21]:
model_pipeline.fit(texts, labelz)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(tokenizer=<function <lambda> at 0x000001E153861430>)),
                ('model', LogisticRegression(random_state=228))])

In [22]:
from sklearn.metrics import precision_score, recall_score, accuracy_score

In [23]:
accuracy_score(y_true=labelz, y_pred = model_pipeline.predict(texts))

0.8173676614038727

# Предобработка и формирование предсказания на тестовой выборке

In [74]:
oren_ochevidec = pd.DataFrame
oren_ochevidec = pd.read_csv("oren_ochevidec .csv", sep=';', encoding="windows-1251") # Загружаем данные

In [75]:
oren_online = pd.DataFrame
oren_online = pd.read_csv("oren_online .csv", sep=';', encoding="windows-1251") 

In [76]:
podslush_oren = pd.DataFrame
podslush_oren = pd.read_csv("podslush_oren .csv", sep=';', encoding="windows-1251") 

In [77]:
oren_online_source = oren_online["source"].values
podslush_oren_source = podslush_oren["source"].values
oren_ochevidec_source = oren_ochevidec["source"].values

oren_online_sender = oren_online["from_id"].values
podslush_oren_sender = podslush_oren["from_id"].values
oren_ochevidec_sender = oren_ochevidec["from_id"].values

oren_online = oren_online["text"].values
podslush_oren = podslush_oren["text"].values
oren_ochevidec = oren_ochevidec["text"].values

In [78]:
def sourcer (source):
    j = 0
    while j < len(source):
        source[j] = source[j][:21]
        j+=1
    
    return source

In [79]:
oren_online_source = sourcer (oren_online_source)
podslush_oren_source = sourcer (podslush_oren_source)
oren_ochevidec_source = sourcer (oren_ochevidec_source)

In [80]:
def stringer(test): # Функция для приведения всех данных в формат string
    i = 0
    while i < len(test):
        test[i]= str(test[i])
        i+=1
    return test

In [81]:
def id_del(test):
    i = 0
    deleted = 0
    output = []
    for j in test:
        while i < len(j):
            if j[i] == ']':
                deleted = 1
                plus = i + 1
                output.append(j[i+1:])
            if i == len(j)-1:
                if deleted == 0:
                    output.append(j)
            i +=1
        i = 0
        deleted = 0
    return output

In [82]:
oren_ochevidec = stringer(oren_ochevidec) # применяем функции 
oren_ochevidec = id_del(oren_ochevidec)


oren_online = stringer(oren_online) # применяем функции 
oren_online = id_del(oren_online)

podslush_oren = stringer(podslush_oren) # применяем функции 
podslush_oren = id_del(podslush_oren)


In [33]:
a = len(oren_ochevidec) + len(oren_online) + len(podslush_oren)
print (" Всего комментариев - > " + str(a))

 Всего комментариев - > 136513


In [34]:
oren_ochevidec

[', богатые тоже скачут ( по ямам)',
 'Богатые тоже плачут',
 'nan',
 'Кошмар! Такие деньги списывают на ремонт дорог каждый год. И как после бомбежки.',
 ', согласен! В Москве водитель автобуса работает на тех же условиях за 80к в месяц, ещё и автобусы удобные, чистые с кондеями и обслуживание централизованное а не в гаражах. Почему сабянин может так, а салмин не может?',
 'Дотации! А что и автобусы есть новые??? А з/п да низкие. Я была в шоке когда узнала что Уборщица, дворник в службе занятости и учитель, специалист, IT-специалист, медсестра, врачи и т.д.  одинаково оцениваются в 15974 руб. Это как себя ненавидеть, чтобы согласиться на такую оплату труда? Стоимость труда дешевая. Так если бы цена труда была выше то и безработица бы ушла. Вот этот вопрос и важно Путину задать. Регионы, которые обеспечивают   почти всем необходимым трудятся почти за спасибо.',
 'Надо не водителей искать, а руководство',
 ', я там колесо на велосипеде убил',
 ', это ещё лужи стали меньше, а то все было

In [35]:
oren_ochevidec_test = model_pipeline.predict(oren_ochevidec)

In [36]:
oren_ochevidec_test

array([0, 1, 0, ..., 0, 1, 0])

In [37]:
oren_online_test = model_pipeline.predict(oren_online)

In [38]:
oren_online_test

array([0, 0, 0, ..., 1, 0, 1])

In [39]:
podslush_oren_test = model_pipeline.predict(podslush_oren)

In [40]:
podslush_oren_test

array([1, 0, 1, ..., 0, 0, 0])

In [41]:
oren_ochevidec[0]

', богатые тоже скачут ( по ямам)'

In [42]:
model_pipeline.predict(["Ненавижу мат анализ"])

array([1])

# Загрузка результатов в Базу Данных (Курсовая Работа)

In [43]:
import pymongo

In [44]:
conn = pymongo.MongoClient()

In [45]:
db = conn.issledovanie

In [46]:
collection = db.comment

In [105]:
new_id = 1
i = 0
while i < len(oren_ochevidec):
    collection.insert_one({"_id" : new_id, "text" : oren_ochevidec[i], "label" : str(oren_ochevidec_test[i]), "post_id" : str (oren_ochevidec_source[i]), "sender" : str(oren_ochevidec_sender[i]), "group" : "oren_ochevidec"})
    new_id += 1
    i+=1

In [106]:
collection_2 = db.comment

In [107]:

i = 0
while i < len(oren_online):
    collection_2.insert_one({"_id" : new_id, "text" : oren_online[i], "label" : str(oren_online_test[i]), "post_id" : str (oren_online_source[i]), "sender" : str(oren_online_sender[i]), "group" : "oren_online"})
    new_id += 1
    i+=1

In [108]:
collection_3 = db.comment

In [109]:

i = 0
while i < len(podslush_oren):
    collection_3.insert_one({"_id" : new_id, "text" : podslush_oren[i], "label" : str(podslush_oren_test[i]), "post_id" : str (podslush_oren_source[i]), "sender" : str(podslush_oren_sender[i]), "group" : "podslush_oren"})
    new_id += 1
    i+=1

# Финальная Аналитика

In [113]:
neg_or_onl = 0
for i in oren_online_test:
    if i == 1:
        neg_or_onl += 1
        
print ("average neg in oren_online is - " + str(neg_or_onl/len(oren_online_test)))

average neg in oren_online is - 0.40255828430289764


In [114]:
neg_or_och = 0
for i in oren_ochevidec_test:
    if i == 1:
        neg_or_och += 1
        
print ("average neg in oren_ochevidec is - " + str(neg_or_och/len(oren_ochevidec_test)))

average neg in oren_ochevidec is - 0.41835934875596265


In [116]:
neg_p_or = 0
for i in podslush_oren_test:
    if i == 1:
        neg_p_or += 1
        
print ("average neg in podslush_oren is - " + str(neg_p_or/len(podslush_oren_test)))

average neg in podslush_oren is - 0.37785597108960794


In [119]:
print ("average negative in vk - " + str ((neg_p_or + neg_or_och + neg_or_onl)/a))

average negative in vk - 0.400115739892904


# Заполнение других коллекций в MongoDB (Курсовая Работа)

In [120]:
soc = db.soc_net

In [121]:
soc.insert_one({"_id" : "vk", "full_name" : "vkontakte" ,"domain" : "vk.com", "all_com_count" : a, "avr_neg" : 0.40})

In [122]:
group = db.group

In [123]:
group.insert_one({"_id" : "oren_ochevidec", "comm_count" : len(oren_ochevidec_test), "aver_neg" : 0.4183})

In [124]:
group.insert_one({"_id" : "oren_online", "comm_count" : len(oren_online_test), "aver_neg" : 0.4025})

In [ ]:
group.insert_one({"_id" : "podslushano_oren", "comm_count" : len(podslush_oren_test), "aver_neg" : 0.3778})